In [134]:
# Add Dependencies
import pandas as pd
import matplotlib.pyplot as plt

In [135]:
# Poiters to the datasets
integ_data_set = 'top_counties.csv'
zhvi_single_family_homes_by_county_url = 'https://files.zillowstatic.com/research/public_csvs/zhvi/County_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_month.csv?t=1720392210'


In [136]:
# Read in the data for the top_counties
integ_top_counties_df = pd.read_csv(
    integ_data_set,
    parse_dates=True,
    infer_datetime_format=True
    )

/var/folders/h3/xtqxxtn9551c8dd44chd8lbc0000gn/T/ipykernel_4593/2501497463.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  integ_top_counties_df = pd.read_csv(


In [137]:
# Read in the ZHVI
zhvi_county_df = pd.read_csv(
    zhvi_single_family_homes_by_county_url,
    parse_dates=True,
    infer_datetime_format=True
    )

/var/folders/h3/xtqxxtn9551c8dd44chd8lbc0000gn/T/ipykernel_4593/63787960.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  zhvi_county_df = pd.read_csv(


In [138]:
print(integ_top_counties_df.head())
print(integ_top_counties_df.tail())

   Unnamed: 0  GeoFips        GeoName      2022      CAGR  Total Growth Rate
0           0     8031     Denver, CO  1.342352  6.732323          18.007194
1           1    30031   Gallatin, MT  1.126722  7.792158          39.259185
2           2    12021    Collier, FL  1.081409  6.919575          23.419708
3           3    41017  Deschutes, OR  1.056533  8.269753          30.814977
4           4    49049       Utah, UT  1.048108  6.837103          35.070734
    Unnamed: 0  GeoFips         GeoName      2022      CAGR  Total Growth Rate
18          18    30063    Missoula, MT  0.827218  6.713997          10.545783
19          19    45019  Charleston, SC  0.820833  7.252412          19.288897
20          20    47037    Davidson, TN  0.813328  7.190962          12.737907
21          21    12035     Flagler, FL  0.811683  6.634026          32.083438
22          22    47187  Williamson, TN  0.810069  7.320579          41.720325


In [139]:
zhvi_county_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,2000-01-31,...,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30
0,3101,0,Los Angeles County,county,CA,CA,"Los Angeles-Long Beach-Anaheim, CA",6,37,223227.570289,...,908470.164191,920227.893039,929015.843494,934548.891149,933373.048335,928068.136342,923865.536315,9.247795e+05,9.284306e+05,9.309001e+05
1,139,1,Cook County,county,IL,IL,"Chicago-Naperville-Elgin, IL-IN-WI",17,31,155001.856326,...,318401.265838,319825.649452,320807.462970,321484.462541,322011.240173,323401.039876,326043.702340,3.292884e+05,3.316914e+05,3.331743e+05
2,1090,2,Harris County,county,TX,TX,"Houston-The Woodlands-Sugar Land, TX",48,201,113426.768603,...,290095.008822,290139.115275,289887.928358,289601.583693,289719.518949,290207.306644,291249.723603,2.923441e+05,2.929599e+05,2.928385e+05
3,2402,3,Maricopa County,county,AZ,AZ,"Phoenix-Mesa-Chandler, AZ",4,13,147617.388832,...,473709.417278,475960.153050,477802.198730,478802.388963,479165.533584,479740.411788,481323.229115,4.832995e+05,4.848863e+05,4.851605e+05
4,2841,4,San Diego County,county,CA,CA,"San Diego-Chula Vista-Carlsbad, CA",6,73,229910.294473,...,951680.611779,961908.232012,968876.125006,972538.358823,974985.819772,980208.456178,989791.934438,1.001835e+06,1.012124e+06,1.017484e+06


In [140]:
# Splitting 'GeoName' into 'RegionName' and 'State'
integ_top_counties_df[['RegionName', 'State']] = integ_top_counties_df['GeoName'].str.split(',', expand=True)
integ_top_counties_df['RegionName'] = integ_top_counties_df['RegionName'].str.strip()
integ_top_counties_df['State'] = integ_top_counties_df['State'].str.strip()
integ_top_counties_df['RegionName'] = integ_top_counties_df['RegionName'] + ' County'
integ_counties = integ_top_counties_df[['RegionName', 'State']]

# Display 
print(integ_counties.head())

         RegionName State
0     Denver County    CO
1   Gallatin County    MT
2    Collier County    FL
3  Deschutes County    OR
4       Utah County    UT


In [141]:
# merge to find matching rows
matched_df = pd.merge(integ_counties, zhvi_county_df, on=['RegionName', 'State'], how='inner')
print(matched_df.shape)
print(matched_df.columns)
print(matched_df.head())



(23, 303)
Index(['RegionName', 'State', 'RegionID', 'SizeRank', 'RegionType',
       'StateName', 'Metro', 'StateCodeFIPS', 'MunicipalCodeFIPS',
       '2000-01-31',
       ...
       '2023-09-30', '2023-10-31', '2023-11-30', '2023-12-31', '2024-01-31',
       '2024-02-29', '2024-03-31', '2024-04-30', '2024-05-31', '2024-06-30'],
      dtype='object', length=303)
         RegionName State  RegionID  SizeRank RegionType StateName  \
0     Denver County    CO       989        96     county        CO   
1   Gallatin County    MT      2299       544     county        MT   
2    Collier County    FL      1638       191     county        FL   
3  Deschutes County    OR      2679       344     county        OR   
4       Utah County    UT       322       105     county        UT   

                        Metro  StateCodeFIPS  MunicipalCodeFIPS  \
0  Denver-Aurora-Lakewood, CO              8                 31   
1                 Bozeman, MT             30                 31   
2     Naples

In [148]:
# Ensure all 'YYYY-MM-DD' columns are kept alongside 'RegionName' and 'State'
columns_to_keep = ['RegionName', 'State'] + [col for col in matched_df.columns if col.endswith('31') or col.endswith('30') or col.endswith('29') or col.endswith('28')]
filtered_df = matched_df[columns_to_keep]
filtered_df['County_State'] = filtered_df['RegionName'] + ', ' + filtered_df['State']
filtered_df.drop(['RegionName', 'State'], axis=1, inplace=True)

# filtered_df.columns
# Move 'County_State' to be the first column
cols = ['County_State'] + [col for col in filtered_df.columns if col != 'County_State']

# Reorder the DataFrame using the new column order
filtered_df = filtered_df[cols]
# print(filtered_df.columns)
filtered_df.head()

/var/folders/h3/xtqxxtn9551c8dd44chd8lbc0000gn/T/ipykernel_4593/3007947652.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['County_State'] = filtered_df['RegionName'] + ', ' + filtered_df['State']
/var/folders/h3/xtqxxtn9551c8dd44chd8lbc0000gn/T/ipykernel_4593/3007947652.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop(['RegionName', 'State'], axis=1, inplace=True)


,County_State,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,2000-08-31,2000-09-30,...,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30
0,"Denver County, CO",188363.475014,189466.504205,190602.880953,193367.950670,196210.983682,199246.923223,201952.807309,204624.575749,207277.773170,...,620530.780336,621513.046641,621975.050484,621267.328216,619763.871817,618542.404492,618522.220482,619088.332300,619020.357167,617569.490636
1,"Gallatin County, MT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,731229.698881,735647.526411,739520.652109,743529.565498,747972.115065,755583.224396,763941.003854,770989.226478,773576.698043,772879.309481
2,"Collier County, FL",190845.457885,191065.876601,192010.824040,193702.464206,195842.883047,197589.083724,199427.162255,201224.100102,203254.421657,...,711559.343580,715724.256888,719785.606207,723502.829825,726651.785804,729131.320977,731364.249989,731868.673262,730262.881223,725597.455523
3,"Deschutes County, OR",153707.717620,154007.399548,154359.502214,155017.199333,155622.253712,156078.028455,156433.451760,156998.215773,157734.867313,...,677827.263624,682512.425123,684813.458042,684649.840690,682218.044658,680635.732868,681042.303921,683505.947191,686223.609764,687962.951948
4,"Utah County, UT",NaN,NaN,NaN,NaN,192970.602342,192757.573730,192596.774045,192583.153208,192988.575854,...,546160.681471,547845.582968,549275.767408,550084.739452,550037.216580,550281.761656,551369.809751,552469.997548,552694.972610,551856.643926


In [143]:
filtered_df.set_index('County_State', inplace=True)

filtered_df.head()

,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,2000-08-31,2000-09-30,2000-10-31,...,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30
County_State,,,,,,,,,,,,,,,,,,,,,
"Denver County, CO",188363.475014,189466.504205,190602.880953,193367.950670,196210.983682,199246.923223,201952.807309,204624.575749,207277.773170,209538.036633,...,620530.780336,621513.046641,621975.050484,621267.328216,619763.871817,618542.404492,618522.220482,619088.332300,619020.357167,617569.490636
"Gallatin County, MT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,731229.698881,735647.526411,739520.652109,743529.565498,747972.115065,755583.224396,763941.003854,770989.226478,773576.698043,772879.309481
"Collier County, FL",190845.457885,191065.876601,192010.824040,193702.464206,195842.883047,197589.083724,199427.162255,201224.100102,203254.421657,204988.505612,...,711559.343580,715724.256888,719785.606207,723502.829825,726651.785804,729131.320977,731364.249989,731868.673262,730262.881223,725597.455523
"Deschutes County, OR",153707.717620,154007.399548,154359.502214,155017.199333,155622.253712,156078.028455,156433.451760,156998.215773,157734.867313,158635.721936,...,677827.263624,682512.425123,684813.458042,684649.840690,682218.044658,680635.732868,681042.303921,683505.947191,686223.609764,687962.951948
"Utah County, UT",NaN,NaN,NaN,NaN,192970.602342,192757.573730,192596.774045,192583.153208,192988.575854,193759.537821,...,546160.681471,547845.582968,549275.767408,550084.739452,550037.216580,550281.761656,551369.809751,552469.997548,552694.972610,551856.643926
